In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [ ]:
import json

In [ ]:
with open('0727_recipe(csv)2.json', 'r') as f:
  rcp_json = json.loads(f.read())

In [ ]:
df2 = pd.json_normalize(rcp_json, record_path=['ingre_list'], meta=['id','name','recipe','tag'])

In [ ]:
df2[:5]

,ingre_name,ingre_count,ingre_unit,id,name,recipe,tag
0,박력분,150,g,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이"
1,강력분,100,g,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이"
2,버터,220,g,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이"
3,소금,1,작은술,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이"
4,물,100,g,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이"


In [ ]:
df2 = df2.groupby('id').agg({'name':'first',
                               'recipe':'first',
                               'tag':'first',
                               'ingre_name':', '.join}).reset_index()

In [ ]:
df2[:5]

,id,name,recipe,tag,ingre_name
0,1011256,바삭하고 고소한 애플파이,[1. 사과는 껍질을 벗겨 씨를 제거하고 8등분 한 후 8mm 두께로 은행잎 모양으...,"사과, 파이","박력분, 강력분, 버터, 소금, 물"
1,1047204,달달한 바나나 강정,"[1. 바나나는 껍질을 벗겨놓고 계란은 풀어놓아요., 2. 바나나를 한 입 크기로 ...",강정,"바나나, 밀가루, 계란, 빵가루"
2,1058215,저렴하고 맛있는 코다리간장조림,"[1. 코다리는 달리 손질할 것이 많이 없어요. 그저 머리부분좀 잘라내고, 꼬리쪽 ...","장조림, 코다리조림","코다리, 청양고추"
3,1058543,타르트지 만들기,[1. 버터크림화해서 타르트지 만들기는 버터를 깍둑 썰기하여 실온에 두어 말랑말랑한...,타르트,"아몬드가루, 박력분, 슈가파우더, 달걀노른자, 우유"
4,1146710,초코칩 소보로머핀 만들기,[1. 소보로 반죽을 먼저 만들어주세요. 살짝 말랑해진 버터 50g은 1cm주사위로...,머핀,"달걀, 박력분, 베이킹파우더, 블루베리"


In [ ]:
def arrToStr(arr):
  return arr.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")

df2['cleand'] = arrToStr(df2['ingre_name']) + ' ' + arrToStr(df2['tag'])

In [ ]:
df2['cleand'].head()

0          박력분  강력분  버터  소금  물 사과  파이
1                바나나  밀가루  계란  빵가루 강정
2                코다리  청양고추 장조림  코다리조림
3    아몬드가루  박력분  슈가파우더  달걀노른자  우유 타르트
4            달걀  박력분  베이킹파우더  블루베리 머핀
Name: cleand, dtype: object

In [ ]:
df2['cleand'].replace('', np.nan, inplace=True)
df2 = df2[df2['cleand'].notna()]
print(len(df2))

9966


In [ ]:
corpus = []
for w in df2['cleand']:
  corpus.append(w.split())

In [ ]:
corpus[:10]

[['박력분', '강력분', '버터', '소금', '물', '사과', '파이'],
 ['바나나', '밀가루', '계란', '빵가루', '강정'],
 ['코다리', '청양고추', '장조림', '코다리조림'],
 ['아몬드가루', '박력분', '슈가파우더', '달걀노른자', '우유', '타르트'],
 ['달걀', '박력분', '베이킹파우더', '블루베리', '머핀'],
 ['닭', '감자', '당근', '양파', '파', '생강', '마늘', '청양고추', '건고추', '찜닭', '치킨'],
 ['물', '우유', '버터', '설탕', '소금', '박력분', '달걀', '요거트'],
 ['박력분', '버터', '설탕', '소금', '노른자', '레몬즙', '타르트'],
 ['포도', '젤라틴', '물', '푸딩'],
 ['박력분', '아몬드가루', '슈가파우더', '달걀흰자', '버터', '소금', '버터링쿠키', '버터쿠키', '아몬드쿠키']]

In [ ]:
word2vec_model = Word2Vec(size=300, window=5, min_count=2, workers=-1)
word2vec_model.build_vocab(corpus)
word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs=15)

(0, 0)

In [ ]:
word2vec_model.save('0728w2v.model')

In [ ]:
df2.to_csv('0728rcp.csv', encoding='utf-8')